# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импорт библиотек и загрузка данных

In [1]:
import numpy as np
import pandas as pd
import torch
import re
import nltk
import warnings

from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import EnglishStemmer
from nltk import word_tokenize,sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import notebook
from pymystem3 import Mystem
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, make_scorer

In [2]:
df_toxic = pd.read_csv('datasets/toxic_comments.csv')
warnings.filterwarnings("ignore")

In [3]:
print(df_toxic.info())
print(df_toxic['toxic'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None
0    143346
1     16225
Name: toxic, dtype: int64


Посмотрим на данные

In [4]:
df_toxic.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
corpus = df_toxic['text'].astype('U').values

In [6]:
for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()

Очистим данные от небуквенных символов

In [7]:
def clear_text(text):
    new_text = re.sub(r'[^a-z]',' ',text)
    split_text = new_text.split()
    return ' '.join(split_text)

Лемматизируем текст

In [8]:
lemm_data = []
for i in range(len(corpus)): 
    clean_text = clear_text(corpus[i])
    tokens = nltk.word_tokenize(clean_text)
    stemmer = EnglishStemmer(ignore_stopwords=False)
    lemm = ' '.join([stemmer.stem(token) for token in tokens])
    lemm_data.append(lemm)

Посмотрим пример, как выглядит текст до лемматизации и после

In [9]:
print('BEFORE:')
print(corpus[0])
print()
print('AFTER:')
print(lemm_data[0])

BEFORE:
explanation
why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27

AFTER:
explan whi the edit made under my usernam hardcor metallica fan were revert they weren t vandal just closur on some gas after i vote at new york doll fac and pleas don t remov the templat from the talk page sinc i m retir now


In [10]:
target = df_toxic['toxic']

In [11]:
lemm_data_train, lemm_data_test, target_train, target_test = train_test_split(
    lemm_data, target, test_size = 0.2, random_state=12345)

In [12]:
print(target_train.value_counts())

0    114670
1     12986
Name: toxic, dtype: int64


In [13]:
count_tf_idf = TfidfVectorizer()
tf_idf_tr = count_tf_idf.fit_transform(lemm_data_train)

In [14]:
tf_idf_te = count_tf_idf.transform(lemm_data_test)

In [15]:
features_train = tf_idf_tr
features_test = tf_idf_te


В ходе подготовки текста комментариев к обучению модели, была проведена его предобработка.

Убраны лишние (небуквенные) символы, проведена лемматизация.

Данные разбиты на обучающую и тестовую выборки.

Признаки векторизованы.

## Обучение

Список `rows`, чтобы сохранять результаты

In [16]:
rows = []

In [17]:
grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
gs = GridSearchCV(LogisticRegression(class_weight='balanced'), param_grid=grid,
                  scoring='f1')

In [18]:
gs.fit(features_train, target_train)
print('param:', gs.best_params_, 'f1_score:', gs.best_score_)

param: {'C': 10.0, 'penalty': 'l2'} f1_score: 0.7695742665949652


In [19]:
lr = {'model':'LogisticRegression', 
      'f1_score': gs.best_score_}
rows.append(lr)

In [20]:
params = {'alpha':[1, 10, 200, 265, 270, 275, 500],
               'penalty':["l1","l2"]}

In [21]:
gs2 =  GridSearchCV(SGDClassifier(class_weight='balanced'),
                    param_grid=params,
                    scoring='f1')

In [22]:
gs2.fit(features_train, target_train)
print('param:', gs2.best_params_, 'f1_score:', gs2.best_score_)

param: {'alpha': 1, 'penalty': 'l2'} f1_score: 0.20403773989438892


In [23]:
sg = {'model':'SGDClassifier', 'f1_score': gs2.best_score_}
rows.append(sg)

In [24]:
ridge_params = {'alpha':[1, 10, 200, 265, 270, 275, 500]}

gs3 =  GridSearchCV(RidgeClassifier(class_weight='balanced'), 
                    param_grid=ridge_params, 
                    scoring='f1')

In [25]:
gs3.fit(features_train, target_train)
print('param:', gs3.best_params_, 'f1_score:', gs3.best_score_)

param: {'alpha': 10} f1_score: 0.7062555282064159


In [26]:
rc = {'model':'RidgeClassifier', 'f1_score': gs3.best_score_}
rows.append(rc)

In [27]:
table_f1 = pd.DataFrame(rows)
print(table_f1.sort_values(by='f1_score', ascending=False).reset_index(drop=True))

                model  f1_score
0  LogisticRegression  0.769574
1     RidgeClassifier  0.706256
2       SGDClassifier  0.204038


В данном разделе было обучено 3 модели: логистическая регрессия, sgd классификатор, основанный на градиентном спуске и гребневая регрессия (Ридж).

Лучший результат показала логистическая регрессия. Этот результат является удовлетворительным, т.к. выше необходимого показателя (0.75).

## Вывод

In [28]:
model = LogisticRegression(class_weight='balanced', C=10.0, penalty='l2')
model.fit(features_train, target_train)
predictions = model.predict(features_test)
f1_tfidf_te = f1_score(target_test, predictions)
print('значение f1 на тестовой выборке:', f1_tfidf_te)

значение f1 на тестовой выборке: 0.7686230248306998


- Проведена предобработка данных: лемматизация и векторизация
- Обучено 3 модели с подбором параметров по лучшей f1_score
- Выбрана линейная регрессия, для которой f1_score > 0.75

Итог работы: Модель сможет классифицировть комментарии на позитивный и негативный

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны